In [2]:
# Import libraries
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import gudhi
from tqdm import tqdm
from persim import PersistenceImager
import invr
import matplotlib as mpl

from pysal.lib import weights
from pysal.lib import weights


from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

from scipy.linalg import solve
from scipy.sparse.linalg import spsolve
import numpy as np

import scipy as sp

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Matplotlib default settings
mpl.rcParams.update(mpl.rcParamsDefault)

In [2]:
def generate_adjacent_counties(dataframe, variable_name):
    """Generate adjacent counties based on given dataframe and variable."""
    filtered_df = dataframe
    adjacent_counties = gpd.sjoin(filtered_df, filtered_df, predicate='intersects', how='left')
    adjacent_counties = adjacent_counties.query('sortedID_left != sortedID_right')
    adjacent_counties = adjacent_counties.groupby('sortedID_left')['sortedID_right'].apply(list).reset_index()
    adjacent_counties.rename(columns={'sortedID_left': 'county', 'sortedID_right': 'adjacent'}, inplace=True)
    adjacencies_list = adjacent_counties['adjacent'].tolist()
    county_list = adjacent_counties['county'].tolist()
    merged_df = pd.merge(adjacent_counties, dataframe, left_on='county', right_on='sortedID', how='left')
    merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')
    return adjacencies_list, merged_df, county_list

In [3]:
def form_simplicial_complex(adjacent_county_list, county_list):
    """Form a simplicial complex based on adjacent counties."""
    max_dimension = 3
    V = invr.incremental_vr([], adjacent_county_list, max_dimension, county_list)
    return V

In [4]:
def create_variable_folders(base_path, variables):
    """Create folders for each variable."""
    for variable in variables:
        os.makedirs(os.path.join(base_path, variable), exist_ok=True)
    print('Done creating folders for each variable')


In [60]:
def generate_generalized_variance(simplices,data_frame, variable_name):

    selected_census = []

    for set in simplices:
        if len(set) == 2 or len(set) == 3:
            for vertice in set:
                if vertice not in selected_census:
                    selected_census.append(vertice)
    
    # print(f'selected census: {selected_census}')

    # print(data_frame.head(3))
    # print(data_frame.columns)

    filtered_census_df = data_frame.loc[data_frame["sortedID"].isin(selected_census)]

    # lattice stored in a geo-table
    wq = weights.contiguity.Queen.from_dataframe(filtered_census_df)
    neighbors_q = wq.neighbors

    QTemp = pd.DataFrame(*wq.full()).astype(int)
    QTemp = QTemp.multiply(-1)

    QTemp.index = filtered_census_df["sortedID"].values
    QTemp.columns = filtered_census_df["sortedID"].values

    # for each row in the fullMatrix dataframe sum the values in the row and take the absolute value and store in the diagonal
    for i in QTemp.index:
        QTemp.loc[i,i] = abs(QTemp.loc[i].sum())

    # print(neighbors_q)
    # print(filtered_census_df.head(3))
    # print(QTemp)


    # Marginal variance code -Multiple clusters

    # transform df to numpy array
    Q = QTemp.to_numpy()

    graph = csr_matrix(QTemp)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)

    print(f"Number of connected components: {n_components}")

    # get the simplices for each component(network)
    component_census = {i: [] for i in range(n_components)}  # Initialize a dictionary for simplices per component
    component_simplices = {i: [] for i in range(n_components)}  # Initialize a dictionary for simplices per component

    # Get the index of the selected census(this way missing census(not selected) will not be included)
    id = QTemp.index.to_list()

    # if there are multiple components in the graph. Assign the simplices to the corresponding component
    if n_components== 1:

        for label, idx in zip(labels, id):
            component_census[label].append(idx)

    elif n_components>1:

        for label, idx in zip(labels, id):
            component_census[label].append(idx)
        
        for simplex in simplices:
            if len(simplex) == 2 or len(simplex) == 3:
                # take the first vertice in the simplex and check component census it belongs to
                vertice = simplex[0]
                for component in component_census:
                    
                    if vertice in component_census[component]:
                        # print(f'vertice {vertice} belongs to component {component}')
                        component_simplices[component].append(simplex)


    data_frame[variable_name+'_marginal_variance'] = None #delete this line

    # assign generalized variance for each n_component
    generalized_variance_dic = {i: [] for i in range(n_components)}  # Initialize a dictionary for each n_component

    for k in range(n_components):
        # print(k)

        # get the length of the labels array where the value is equal to i
        # print(len(labels[labels == k]))

        if len(labels[labels==k])==1:

            # get the index of the label
            index = np.where(labels==k)[0][0]
            # print(index)

            #this part is not written becase: does not exists

            # # get the index from Q_df
            # print(Q_df.index[index])

            # print(f"Region {k} is an isolated region")
            # print(f"Marginal Variances with FIPS: {list(zip(Qmatrix[0].index, marginal_variances))}")
            generalized_variance_dic[k] = 1  #CHECK THIS VALUE
        else:
            # print(f"Region {k} is a connected region")

            # get the location index to an array 
            index = np.where(labels == k)
            # print(index)

            # Extract the submatrix
            QQ = Q[np.ix_(index[0], index[0])]

            # print(QQ)

            n = QQ.shape[0]

            
            Q_jitter = QQ + sp.sparse.diags(np.ones(n)) * max(QQ.diagonal()) * np.sqrt(

                np.finfo(np.float64).eps

            )


            # inverse of precision (Q) is cov

            Q_perturbed = sp.sparse.csc_array(Q_jitter)

            b = sp.sparse.identity(n, format='csc')

            sigma = spsolve(Q_perturbed, b)


            # V \in Null(Q)

            V = np.ones(n)  # from pg. 6

            W = sigma @ V.T  # \Sigma * B in 3.17

            Q_inv = sigma - np.outer(W * solve(V @ W, np.ones(1)), W.T)

            # grabbing diag of cov gives var and

            # arithmetic mean in log-space becomes geometric mean after exp

            generalized_variance = np.exp(np.mean(np.log(np.diag(Q_inv))))  # equation in the paper use daba as 1
            # generalized_variance = np.exp(np.sum(np.log(np.diag(Q_inv))) / n) #same as above

            generalized_variance_dic[k] = generalized_variance

            # print(f"Generalized Variance: {generalized_variance}")

    return generalized_variance_dic, component_census, component_simplices

In [ ]:
def process_state(state, selected_variables, selected_variables_with_censusinfo, base_path, PERSISTENCE_IMAGE_PARAMS, INFINITY):
    """Process data for a given state."""
    svi_od_path = os.path.join(data_path, state, state + '.shp')
    svi_od = gpd.read_file(svi_od_path)
    # # for variable in selected_variables:
    #     # svi_od = svi_od[svi_od[variable] != -999]

        
    svi_od_filtered_state = svi_od[selected_variables_with_censusinfo].reset_index(drop=True)

    # Get the unique counties
    unique_county_stcnty = svi_od_filtered_state['STCNTY'].unique()

    for county_stcnty in unique_county_stcnty:
        # Filter the dataframe to include only the current county
        county_svi_df = svi_od_filtered_state[svi_od_filtered_state['STCNTY'] == county_stcnty]

        # print("County")
        # print(county_svi_df)
    
        for variable_name in selected_variables:
            df_one_variable = county_svi_df[['STCNTY','FIPS', variable_name, 'geometry']]
            df_one_variable = df_one_variable.sort_values(by=variable_name)
            df_one_variable['sortedID'] = range(len(df_one_variable))
            df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
            df_one_variable.crs = "EPSG:3395"

            adjacencies_list, adjacent_counties_df, county_list = generate_adjacent_counties(df_one_variable, variable_name)
            adjacent_counties_dict = dict(zip(adjacent_counties_df['county'], adjacent_counties_df['adjacent']))
            county_list = adjacent_counties_df['county'].tolist()
            simplices = form_simplicial_complex(adjacent_counties_dict, county_list)

            print(f'length of simplices: {len(simplices)}')

            if len(simplices)==0:
                print(f'No simplices for {variable_name} in {county_stcnty}')
                print(df_one_variable)
            else:
                print(f'State: {state}')
                print(f'County: {county_stcnty}')
                print(f'County: {variable_name}')

                # print("Simplices",simplices)

                generalized_variance = generate_generalized_variance(simplices=simplices,data_frame=df_one_variable, variable_name=variable_name)

                print(f'Generalized Variance: {generalized_variance}\n')

                # print(f'Generalized Variance: {generalized_variance}')

                # Generate persistence images based on the generalized variance
                # generate_persistence_images(simplices, df_one_variable, variable_name, county_stcnty, base_path, PERSISTENCE_IMAGE_PARAMS, generalized_variance)

            # break

        # break

In [55]:
def generate_persistence_images(simplices, df_one_variable, variable_name, county_stcnty, base_path, PERSISTENCE_IMAGE_PARAMS, generalized_variance_dic, component_census, component_simplices):
    """Generate persistence images."""

    if len(generalized_variance_dic)==1:

        generalized_variance = list(generalized_variance_dic.values())[0]

        st = gudhi.SimplexTree()
        st.set_dimension(2)

        for simplex in simplices:
            if len(simplex) == 1:
                st.insert([simplex[0]], filtration=0.0)

        for simplex in simplices:
            if len(simplex) == 2:
                last_simplex = simplex[-1]
                filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
                st.insert(simplex, filtration=filtration_value)

        for simplex in simplices:
            if len(simplex) == 3:
                last_simplex = simplex[-1]
                filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
                st.insert(simplex, filtration=filtration_value)

        st.compute_persistence()
        persistence = st.persistence()

        intervals_dim0 = st.persistence_intervals_in_dimension(0)
        intervals_dim1 = st.persistence_intervals_in_dimension(1)
        pdgms = [[birth, death] for birth, death in intervals_dim1 if death < np.inf]

        # add interval dim 0  to the pdgms
        for birth, death in intervals_dim0:
            if death < np.inf:
                pdgms.append([birth, death])
            # elif death == np.inf:
                # pdgms.append([birth, INFINITY])
            

        save_path = os.path.join(base_path, variable_name, county_stcnty)

        if len(pdgms) > 0:
            
            # print(f'Processing {variable_name} for {county_stcnty}')
            # print(f'Number of persistence diagrams: {len(pdgms)}')
            # print(intervals_dim1)
            # for i in range(len(intervals_dim1)):
            #     if np.isinf(pdgms[i][1]):
            #         pdgms[i][1] = 1
            #     if np.isinf(pdgms[i][0]):
            #         pdgms[i][0] = 1

            pimgr = PersistenceImager(pixel_size=0.01)
            pimgr.fit(pdgms)

            pimgr.pixel_size = PERSISTENCE_IMAGE_PARAMS['pixel_size']
            pimgr.birth_range = PERSISTENCE_IMAGE_PARAMS['birth_range']
            pimgr.pers_range = PERSISTENCE_IMAGE_PARAMS['pers_range']
            # pimgr.kernel_params = PERSISTENCE_IMAGE_PARAMS['kernel_params']
            pimgr.kernel_params =  {'sigma': generalized_variance}


            pimgs = pimgr.transform(pdgms)
            pimgs = np.rot90(pimgs, k=1) 

            # np.save(save_path, pimgs)

            plt.figure(figsize=(2.4, 2.4))
            plt.imshow(pimgs, cmap='viridis')  # Assuming 'viridis' colormap, change as needed
            plt.axis('off')  # Turn off axis
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust subplot parameters to remove borders
            
            plt.savefig(f'{base_path}/{variable_name}/{county_stcnty}.png')
            plt.close()
    elif len(generalized_variance_dic)>1:

        # each sub network will generate a separate persistence image
        per_images_per_subcomponent = []

        for key in component_census.keys():
            # print(key)
            
            generalized_variance = generalized_variance_dic[key]
            simplices_sub = component_simplices[key]
            census_sub = component_census[key]
            # print(f'Generalized Variance: {generalized_variance}')
            # print(f'Simplices: {simplices_sub}')
            # print(f'Census: {census_sub}')


            # Generate persistence images based on the generalized variance

            st = gudhi.SimplexTree()
            st.set_dimension(2)

            for simplex in census_sub:
                # print(simplex)
            #     # if len(simplex) == 1:
                st.insert([simplex], filtration=0.0)

            for simplex in simplices_sub:
                if len(simplex) == 2:
                    last_simplex = simplex[-1]
                    filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
                    st.insert(simplex, filtration=filtration_value)

            for simplex in simplices_sub:
                if len(simplex) == 3:
                    last_simplex = simplex[-1]
                    filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
                    st.insert(simplex, filtration=filtration_value)

            st.compute_persistence()
            persistence = st.persistence()

            intervals_dim0 = st.persistence_intervals_in_dimension(0)
            intervals_dim1 = st.persistence_intervals_in_dimension(1)
            pdgms = [[birth, death] for birth, death in intervals_dim1 if death < np.inf]

            # add interval dim 0  to the pdgms
            for birth, death in intervals_dim0:
                if death < np.inf:
                    pdgms.append([birth, death])
                # elif death == np.inf:
                    # pdgms.append([birth, INFINITY])
                

            # save_path = os.path.join(base_path, variable_name, county_stcnty)

            if len(pdgms) > 0:
                
                # print(f'Processing {variable_name} for {county_stcnty}')
                # print(f'Number of persistence diagrams: {len(pdgms)}')
                # print(intervals_dim1)
                # for i in range(len(intervals_dim1)):
                #     if np.isinf(pdgms[i][1]):
                #         pdgms[i][1] = 1
                #     if np.isinf(pdgms[i][0]):
                #         pdgms[i][0] = 1

                pimgr = PersistenceImager(pixel_size=0.01)
                pimgr.fit(pdgms)

                pimgr.pixel_size = PERSISTENCE_IMAGE_PARAMS['pixel_size']
                pimgr.birth_range = PERSISTENCE_IMAGE_PARAMS['birth_range']
                pimgr.pers_range = PERSISTENCE_IMAGE_PARAMS['pers_range']
                # pimgr.kernel_params = PERSISTENCE_IMAGE_PARAMS['kernel_params']
                pimgr.kernel_params =  {'sigma': generalized_variance}

                pimgs = pimgr.transform(pdgms)
                pimgs = np.rot90(pimgs, k=1) 
                per_images_per_subcomponent.append(pimgs)

        final_pimgr = np.sum(per_images_per_subcomponent, axis=0)
        print(final_pimgr.shape)

        plt.figure(figsize=(2.4, 2.4))
        plt.imshow(final_pimgr, cmap='viridis')  # Assuming 'viridis' colormap, change as needed
        plt.axis('off')  # Turn off axis
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust subplot parameters to remove borders
        
        plt.show()


In [ ]:
# length of simplices: 79
# State: FL
# County: 12087
# County: EP_POV
# Number of connected components: 3

In [ ]:
# State: NC
# County: 37055
# County: EP_GROUPQ
# Number of connected components: 2
# Generalized Variance: {0: 0.46338322996371606, 1: 0.25}

In [8]:
data_path = '/home/h6x/git_projects/ornl-svi-data-processing/processed_data/SVI/SVI2018_MIN_MAX_SCALED_MISSING_REMOVED'
selected_variables = [
         'EP_POV','EP_UNEMP', 'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65', 'EP_AGE17', 'EP_DISABL', 
        'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ'
    ]

In [9]:
selected_variables_with_censusinfo = ['FIPS', 'STCNTY'] + selected_variables + ['geometry']

In [10]:
state = 'NC'

In [11]:
svi_od_path = os.path.join(data_path, state, state + '.shp')
svi_od = gpd.read_file(svi_od_path)

In [12]:
county_stcnty = '37055'

In [13]:
svi_od_filtered_state = svi_od[selected_variables_with_censusinfo].reset_index(drop=True)

In [14]:
# Filter the dataframe to include only the current county
county_svi_df = svi_od_filtered_state[svi_od_filtered_state['STCNTY'] == county_stcnty]

In [15]:
county_svi_df

,FIPS,STCNTY,EP_POV,EP_UNEMP,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,EP_MINRTY,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,geometry
231,37055970101,37055,0.048,0.017,0.029677,0.042,0.396685,0.250704,0.132546,0.032,0.000000,0.064,0.051,0.010,0.006,0.006,0.000000,"POLYGON ((-75.79544 36.22711, -75.77707 36.230..."
232,37055970102,37055,0.040,0.069,0.033548,0.091,0.216575,0.256338,0.149606,0.062,0.004594,0.024,0.021,0.030,0.023,0.041,0.000000,"POLYGON ((-75.74282 36.09292, -75.73287 36.095..."
233,37055970200,37055,0.106,0.084,0.107097,0.198,0.161326,0.219718,0.128609,0.023,0.079632,0.081,0.039,0.034,0.000,0.023,0.000000,"POLYGON ((-75.70794 36.04744, -75.69384 36.054..."
234,37055970300,37055,0.096,0.032,0.061935,0.187,0.179006,0.305634,0.124672,0.095,0.059724,0.129,0.054,0.086,0.047,0.010,0.000000,"POLYGON ((-75.72983 36.00729, -75.72708 36.010..."
235,37055970400,37055,0.083,0.027,0.025806,0.108,0.233149,0.230986,0.116798,0.117,0.004594,0.097,0.040,0.012,0.004,0.055,0.028028,"POLYGON ((-75.67020 35.98438, -75.64860 35.993..."
236,37055970502,37055,0.187,0.119,0.149677,0.171,0.246409,0.250704,0.275591,0.086,0.015314,0.093,0.040,0.058,0.044,0.068,0.003003,"POLYGON ((-75.75172 35.19259, -75.73950 35.195..."
237,37055970601,37055,0.052,0.049,0.118710,0.139,0.200000,0.295775,0.241470,0.047,0.027565,0.231,0.086,0.098,0.038,0.040,0.011011,"POLYGON ((-75.72681 35.93584, -75.71827 35.939..."
1432,37055970501,37055,0.067,0.015,0.211613,0.236,0.194475,0.290141,0.240157,0.006,0.000000,0.171,0.000,0.283,0.069,0.023,0.000000,"POLYGON ((-76.01302 35.67007, -76.00928 35.675..."
1433,37055970602,37055,0.050,0.014,0.129032,0.232,0.186740,0.361972,0.216535,0.065,0.045942,0.187,0.022,0.216,0.019,0.016,0.000000,"POLYGON ((-75.67590 35.88298, -75.67171 35.884..."


In [40]:
county_svi_df.shape

(9, 18)

In [16]:
variable_name = 'EP_GROUPQ'

In [17]:
df_one_variable = county_svi_df[['STCNTY','FIPS', variable_name, 'geometry']]
df_one_variable = df_one_variable.sort_values(by=variable_name)
df_one_variable['sortedID'] = range(len(df_one_variable))
df_one_variable = gpd.GeoDataFrame(df_one_variable, geometry='geometry')
df_one_variable.crs = "EPSG:3395"

adjacencies_list, adjacent_counties_df, county_list = generate_adjacent_counties(df_one_variable, variable_name)
adjacent_counties_dict = dict(zip(adjacent_counties_df['county'], adjacent_counties_df['adjacent']))
county_list = adjacent_counties_df['county'].tolist()
simplices = form_simplicial_complex(adjacent_counties_dict, county_list)

print(f'length of simplices: {len(simplices)}')

length of simplices: 14


In [44]:
len(county_list)

7

In [45]:
county_list

[0, 1, 2, 3, 5, 7, 8]

In [43]:
adjacent_counties_dict

{0: [1], 1: [3, 2, 0], 2: [3, 1], 3: [8, 2, 1], 5: [7], 7: [5], 8: [3]}

In [41]:
df_one_variable

,STCNTY,FIPS,EP_GROUPQ,geometry,sortedID
231,37055,37055970101,0.000000,"POLYGON ((-75.795 36.227, -75.777 36.231, -75....",0
232,37055,37055970102,0.000000,"POLYGON ((-75.743 36.093, -75.733 36.096, -75....",1
233,37055,37055970200,0.000000,"POLYGON ((-75.708 36.047, -75.694 36.054, -75....",2
234,37055,37055970300,0.000000,"POLYGON ((-75.730 36.007, -75.727 36.011, -75....",3
1432,37055,37055970501,0.000000,"POLYGON ((-76.013 35.670, -76.009 35.676, -76....",4
1433,37055,37055970602,0.000000,"POLYGON ((-75.676 35.883, -75.672 35.885, -75....",5
236,37055,37055970502,0.003003,"POLYGON ((-75.752 35.193, -75.739 35.196, -75....",6
237,37055,37055970601,0.011011,"POLYGON ((-75.727 35.936, -75.718 35.940, -75....",7
235,37055,37055970400,0.028028,"POLYGON ((-75.670 35.984, -75.649 35.993, -75....",8


In [18]:
simplices

[[0],
 [1],
 [0, 1],
 [2],
 [1, 2],
 [3],
 [1, 3],
 [2, 3],
 [1, 2, 3],
 [5],
 [7],
 [5, 7],
 [8],
 [3, 8]]

In [ ]:
# generalized variance code

In [19]:
selected_census = []

for set in simplices:
    if len(set) == 2 or len(set) == 3:
        for vertice in set:
            if vertice not in selected_census:
                selected_census.append(vertice)

In [20]:
selected_census

[0, 1, 2, 3, 5, 7, 8]

In [21]:
filtered_census_df = df_one_variable.loc[df_one_variable["sortedID"].isin(selected_census)]

In [22]:
# lattice stored in a geo-table
wq = weights.contiguity.Queen.from_dataframe(filtered_census_df)
neighbors_q = wq.neighbors

/home/h6x/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  W.__init__(self, neighbors, ids=ids, **kw)


In [23]:
QTemp = pd.DataFrame(*wq.full()).astype(int)
QTemp = QTemp.multiply(-1)

QTemp.index = filtered_census_df["sortedID"].values
QTemp.columns = filtered_census_df["sortedID"].values

# for each row in the fullMatrix dataframe sum the values in the row and take the absolute value and store in the diagonal
for i in QTemp.index:
    QTemp.loc[i,i] = abs(QTemp.loc[i].sum())

# print(neighbors_q)
# print(filtered_census_df.head(3))
# print(QTemp)


# Marginal variance code -Multiple clusters

# transform df to numpy array
Q = QTemp.to_numpy()

In [24]:
QTemp

,0,1,2,3,5,7,8
0,1,-1,0,0,0,0,0
1,-1,3,-1,-1,0,0,0
2,0,-1,2,-1,0,0,0
3,0,-1,-1,3,0,0,-1
5,0,0,0,0,1,-1,0
7,0,0,0,0,-1,1,0
8,0,0,0,-1,0,0,1


In [46]:
QTemp.shape

(7, 7)

In [48]:
# get the index of the QTemp
QTemp.index.to_list()

[0, 1, 2, 3, 5, 7, 8]

In [39]:
Q

array([[ 1, -1,  0,  0,  0,  0,  0],
       [-1,  3, -1, -1,  0,  0,  0],
       [ 0, -1,  2, -1,  0,  0,  0],
       [ 0, -1, -1,  3,  0,  0, -1],
       [ 0,  0,  0,  0,  1, -1,  0],
       [ 0,  0,  0,  0, -1,  1,  0],
       [ 0,  0,  0, -1,  0,  0,  1]])

In [25]:
graph = csr_matrix(QTemp)
n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)

print(f"Number of connected components: {n_components}")

Number of connected components: 2


In [26]:
labels

array([0, 0, 0, 0, 1, 1, 0], dtype=int32)

In [27]:
len(labels)

7

In [28]:
# Step 6: Group vertices by their component labels
component_simplices = {i: [] for i in range(n_components)}  # Initialize a dictionary for simplices per component


In [51]:
component_census = {i: [] for i in range(n_components)}  # Initialize a dictionary for simplices per component

In [30]:
component_census

{0: [], 1: []}

In [31]:
component_simplices

{0: [], 1: []}

In [37]:
labels

array([0, 0, 0, 0, 1, 1, 0], dtype=int32)

In [38]:
len(labels)

7

In [50]:
id = QTemp.index.to_list()

In [52]:
# loop through labels and id same time
for label, idx in zip(labels, id):
    component_census[label].append(idx)

In [53]:
component_census

{0: [0, 1, 2, 3, 8], 1: [5, 7]}

In [32]:
for idx, label in enumerate(labels):

    # print(idx, label)
    component_census[label].append(idx)

In [33]:
component_census

{0: [0, 1, 2, 3, 6], 1: [4, 5]}

In [36]:
for simplex in simplices:

    if len(simplex) == 2 or len(simplex) == 3:

        # take the first vertice in the simplex and check component census it belongs to
        vertice = simplex[0]
        print("vertice",vertice)

        for component in component_census:
            # print("component",component)

            if vertice in component_census[component]:
                print(f'vertice {vertice} belongs to component {component}')
                # component_simplices[component].append(simplex)

            #     component_simplices[component].append(simplex)
        # break

    

vertice 0
vertice 0 belongs to component 0
vertice 1
vertice 1 belongs to component 0
vertice 1
vertice 1 belongs to component 0
vertice 2
vertice 2 belongs to component 0
vertice 1
vertice 1 belongs to component 0
vertice 5
vertice 5 belongs to component 1
vertice 3
vertice 3 belongs to component 0


In [35]:
component_simplices

{0: [[0, 1], [1, 2], [1, 3], [2, 3], [1, 2, 3], [3, 8]], 1: [[5, 7]]}

In [ ]:
component_census[0]

In [ ]:
for simplex in simplices:
    if len(simplex) == 1:
        # st.insert([simplex[0]], filtration=0.0)

In [ ]:
component_simplices[0]

In [ ]:
for simplex in component_simplices[0]:
    if len(simplex) == 2:
        print(simplex)
        last_simplex = simplex[-1]
        print(last_simplex)
        filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
        print(filtration_value)


In [ ]:
for simplex in simplices:
    if len(simplex) == 2:
        last_simplex = simplex[-1]
        filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
        # st.insert(simplex, filtration=filtration_value)



In [ ]:
for simplex in simplices:
    if len(simplex) == 3:
        last_simplex = simplex[-1]
        filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
        # st.insert(simplex, filtration=filtration_value)

In [ ]:
component_census

In [ ]:
len(component_census)

In [61]:
generalized_variance_dic, component_census, component_simplices = generate_generalized_variance(simplices=simplices,data_frame=df_one_variable, variable_name=variable_name)

Number of connected components: 2


/home/h6x/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  W.__init__(self, neighbors, ids=ids, **kw)


In [62]:
list(generalized_variance_dic.values())[0]

0.46338322996371606

In [63]:
generalized_variance_dic

{0: 0.46338322996371606, 1: 0.25}

In [64]:
component_census

{0: [0, 1, 2, 3, 8], 1: [5, 7]}

In [65]:
component_simplices

{0: [[0, 1], [1, 2], [1, 3], [2, 3], [1, 2, 3], [3, 8]], 1: [[5, 7]]}

In [66]:
# get the keys of the dictionary component_census

In [67]:
for key in component_census.keys():
    print(key)

0
1


In [69]:
PERSISTENCE_IMAGE_PARAMS = {
        'pixel_size': 0.001,
        'birth_range': (0.0, 1.00),
        'pers_range': (0.0, 0.40),
        'kernel_params': {'sigma': 0.0003}
    }

In [73]:
per_images_per_subcomponent = []

for key in component_census.keys():
    print(key)
    
    generalized_variance = generalized_variance_dic[key]
    simplices_sub = component_simplices[key]
    census_sub = component_census[key]
    print(f'Generalized Variance: {generalized_variance}')
    print(f'Simplices: {simplices_sub}')
    print(f'Census: {census_sub}')


    # Generate persistence images based on the generalized variance

    st = gudhi.SimplexTree()
    st.set_dimension(2)

    for simplex in census_sub:
        print(simplex)
    #     # if len(simplex) == 1:
        st.insert([simplex], filtration=0.0)

    for simplex in simplices_sub:
        if len(simplex) == 2:
            last_simplex = simplex[-1]
            filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
            st.insert(simplex, filtration=filtration_value)

    for simplex in simplices_sub:
        if len(simplex) == 3:
            last_simplex = simplex[-1]
            filtration_value = df_one_variable.loc[df_one_variable['sortedID'] == last_simplex, variable_name].values[0]
            st.insert(simplex, filtration=filtration_value)

    st.compute_persistence()
    persistence = st.persistence()

    intervals_dim0 = st.persistence_intervals_in_dimension(0)
    intervals_dim1 = st.persistence_intervals_in_dimension(1)
    pdgms = [[birth, death] for birth, death in intervals_dim1 if death < np.inf]

    # add interval dim 0  to the pdgms
    for birth, death in intervals_dim0:
        if death < np.inf:
            pdgms.append([birth, death])
        # elif death == np.inf:
            # pdgms.append([birth, INFINITY])
        

    # save_path = os.path.join(base_path, variable_name, county_stcnty)

    if len(pdgms) > 0:
        
        # print(f'Processing {variable_name} for {county_stcnty}')
        # print(f'Number of persistence diagrams: {len(pdgms)}')
        # print(intervals_dim1)
        # for i in range(len(intervals_dim1)):
        #     if np.isinf(pdgms[i][1]):
        #         pdgms[i][1] = 1
        #     if np.isinf(pdgms[i][0]):
        #         pdgms[i][0] = 1

        pimgr = PersistenceImager(pixel_size=0.01)
        pimgr.fit(pdgms)

        pimgr.pixel_size = PERSISTENCE_IMAGE_PARAMS['pixel_size']
        pimgr.birth_range = PERSISTENCE_IMAGE_PARAMS['birth_range']
        pimgr.pers_range = PERSISTENCE_IMAGE_PARAMS['pers_range']
        pimgr.kernel_params =  {'sigma': generalized_variance}

        pimgs = pimgr.transform(pdgms)
        pimgs = np.rot90(pimgs, k=1) 
        per_images_per_subcomponent.append(pimgs)

        print(pimgs.shape)




0
Generalized Variance: 0.46338322996371606
Simplices: [[0, 1], [1, 2], [1, 3], [2, 3], [1, 2, 3], [3, 8]]
Census: [0, 1, 2, 3, 8]
0
1
2
3
8
(400, 1000)
1
Generalized Variance: 0.25
Simplices: [[5, 7]]
Census: [5, 7]
5
7
(400, 1000)


In [ ]:
len(per_images_per_subcomponent)

In [ ]:
per_images_per_subcomponent[0].shape

In [ ]:
per_images_per_subcomponent[1].shape

In [ ]:
per_images_per_subcomponent[2].shape

In [ ]:
type(per_images_per_subcomponent[2])

In [ ]:
type(per_images_per_subcomponent)

In [ ]:
import numpy as np

combined_array = np.concatenate(per_images_per_subcomponent, axis=0)

In [ ]:
combined_array.shape

In [ ]:
A = per_images_per_subcomponent[0] + per_images_per_subcomponent[1] + per_images_per_subcomponent[2]

In [ ]:
A.shape

In [ ]:
print(per_images_per_subcomponent[0][12][40])
print(per_images_per_subcomponent[1][12][40])
print(per_images_per_subcomponent[2][12][40])

In [ ]:
B = np.sum(per_images_per_subcomponent, axis=0)

In [ ]:
B.shape

In [3]:
dff = gpd.read_file("/home/h6x/git_projects/ornl-svi-data-processing/processed_data/SVI/SVI2018_NOT_SCALED_MISSING_REMOVED/NY/NY.shp")

In [4]:
dff.shape

(4821, 127)

In [5]:
dff.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Leng,Shape_Area,geometry
0,36,NEW YORK,NY,36001,Albany,36001000100,"Census Tract 1, Albany County, New York",0.914079,2022,218,...,0,3,128,57,6.3,2.8,3031,0.070750,0.000286,"POLYGON ((-73.74506 42.67228, -73.74374 42.677..."
1,36,NEW YORK,NY,36001,Albany,36001000403,"Census Tract 4.03, Albany County, New York",1.211858,4236,365,...,0,0,133,104,3.1,2.5,11514,0.111548,0.000344,"POLYGON ((-73.82108 42.67858, -73.81916 42.680..."
2,36,NEW YORK,NY,36001,Albany,36001001700,"Census Tract 17, Albany County, New York",0.527898,4486,406,...,0,0,117,71,2.7,1.6,5072,0.059361,0.000151,"POLYGON ((-73.80454 42.66263, -73.80212 42.662..."
3,36,NEW YORK,NY,36001,Albany,36001001801,"Census Tract 18.01, Albany County, New York",3.107938,6993,533,...,0,1,77,66,1.1,0.9,4699,0.196286,0.000887,"POLYGON ((-73.84719 42.64865, -73.84226 42.652..."
4,36,NEW YORK,NY,36001,Albany,36001001802,"Census Tract 18.02, Albany County, New York",0.735887,4286,379,...,0,0,93,68,2.2,1.6,1683,0.072816,0.000211,"POLYGON ((-73.82955 42.66075, -73.82684 42.662..."


In [7]:
# get the count of each unique STCNTY
dff['STCNTY'].value_counts().to_dict()

{'36047': 749,
 '36081': 644,
 '36005': 332,
 '36103': 322,
 '36061': 279,
 '36059': 278,
 '36029': 233,
 '36119': 219,
 '36055': 190,
 '36067': 140,
 '36085': 107,
 '36071': 79,
 '36027': 78,
 '36001': 75,
 '36065': 71,
 '36087': 65,
 '36063': 60,
 '36007': 55,
 '36091': 50,
 '36111': 47,
 '36093': 43,
 '36083': 42,
 '36013': 35,
 '36101': 30,
 '36075': 29,
 '36089': 27,
 '36069': 25,
 '36105': 24,
 '36045': 24,
 '36109': 23,
 '36117': 22,
 '36021': 21,
 '36015': 21,
 '36009': 20,
 '36113': 19,
 '36011': 19,
 '36079': 19,
 '36043': 19,
 '36019': 18,
 '36077': 17,
 '36115': 17,
 '36057': 16,
 '36053': 16,
 '36035': 15,
 '36037': 15,
 '36033': 14,
 '36039': 14,
 '36031': 13,
 '36025': 13,
 '36051': 13,
 '36023': 12,
 '36003': 12,
 '36017': 12,
 '36121': 11,
 '36099': 10,
 '36107': 10,
 '36073': 10,
 '36095': 7,
 '36049': 7,
 '36097': 5,
 '36123': 5,
 '36041': 4}